# Explore some data


# GENIE Datasets

### Synapse Links

* GENIE 12.0 public https://www.synapse.org/#!Synapse:syn32309524
* GENIE 13.3 consortium https://www.synapse.org/#!Synapse:syn36709873

  
### Data Format Explanations

* https://docs.gdc.cancer.gov/Data/File_Formats/MAF_Format/
* https://docs.cbioportal.org/file-formats/#mutation-data
* https://docs.cbioportal.org/file-formats/#discrete-copy-number-data
* https://docs.cbioportal.org/file-formats/#segmented-data
  


# Neurofibromatosis Review

* [Current Understanding of Neurofibromatosis Type 1, 2, and Schwannomatosis](https://pubmed.ncbi.nlm.nih.gov/34072574/)
* [Neurofibromatosis type 1-associated tumours: Their somatic mutational spectrum and pathogenesis](https://link.springer.com/article/10.1186/1479-7364-5-6-623)

In [ ]:
from collections import Counter
import itertools
import json
import math
from pathlib import Path
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from tqdm import tqdm

In [ ]:
from nextgenlp import genie
from nextgenlp import genie_constants
from nextgenlp import synapse

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
pd.set_option('display.max_columns', 200)

In [ ]:
GENIE_VERSION = "genie-12.0-public"
#GENIE_VERSION = "genie-13.3-consortium"

# Synapse File Paths

If you are not using the python client to sync synapse data then replace these file paths with the file paths on your local system. 

In [ ]:
syn_file_paths = synapse.get_file_name_to_path(genie_version=GENIE_VERSION)
syn_file_paths

# GENIE - Gene Panels

In [ ]:
df_gp_tall = genie.read_gene_panels(syn_file_paths["gene_panels"], style="tall")
df_gp_wide = genie.read_gene_panels(syn_file_paths["gene_panels"], style="wide")

In [ ]:
df_gp_tall

In [ ]:
df_gp_wide

# GENIE - Assay Information

In [ ]:
df_asi = genie.read_assay_information(syn_file_paths['assay_information'])

In [ ]:
df_asi

# GENIE - Genomic Information

In [ ]:
df_gi = genie.read_genomic_information(syn_file_paths['genomic_information'])

In [ ]:
df_gi

# GENIE - Data Fusions

In [ ]:
df_fu = genie.read_data_fusions(syn_file_paths['data_fusions'])

In [ ]:
df_fu

# GENIE - Data Gene Matrix

In [ ]:
df_dgm = genie.read_data_gene_matrix(syn_file_paths['data_gene_matrix'])

In [ ]:
df_dgm

# GENIE - Data Clinical Patient

In [ ]:
df_dcp = genie.read_clinical_patient(syn_file_paths['data_clinical_patient'])

In [ ]:
df_dcp

# GENIE - Data Clinical Sample

In [ ]:
df_dcs = genie.read_clinical_sample(syn_file_paths['data_clinical_sample'])

In [ ]:
df_dcs

### Samples per Patient

In [ ]:
df_plt = (
    df_dcs.groupby("PATIENT_ID")
    .size()
    .sort_values()
    .to_frame("samples per patient")
    .reset_index()
)
df_plt

In [ ]:
sns.histplot(df_plt, x="samples per patient", discrete=True, log_scale=(False, False))

# GENIE - Data Mutations Extended

In [ ]:
df_dme = genie.read_mutations_extended(syn_file_paths["data_mutations_extended"])

In [ ]:
df_dme

### Variant Classification Distribution

In [ ]:
df_plt = (
    df_dme.groupby("Variant_Classification")
    .size()
    .sort_values()
    .to_frame("count")
    .reset_index()
)
df_plt

In [ ]:
g = sns.barplot(data=df_plt, x="count", y="Variant_Classification", color="grey")
# g.set_xscale("log")
g

### Variants per Sample

In [ ]:
df_plt = (
    df_dme.groupby("Tumor_Sample_Barcode")
    .size()
    .sort_values()
    .to_frame("variants per sample")
    .reset_index()
)
df_plt

In [ ]:
sns.histplot(df_plt, x="variants per sample", log_scale=(True, True))

In [ ]:
sns.histplot(
    df_plt[df_plt["variants per sample"] < 100],
    x="variants per sample",
    log_scale=(False, True),
)

# Samples with no mutations

In [ ]:
sample_ids = set(df_dcs.index) - set(df_dme['Tumor_Sample_Barcode'])

In [ ]:
df_dcs.loc[sample_ids]

# GENIE - Merge Patient/Sample/Mutations

In [ ]:
df_psm = genie.read_pat_sam_mut(
    syn_file_paths["data_clinical_patient"],
    syn_file_paths["data_clinical_sample"],
    syn_file_paths["data_mutations_extended"],
)

In [ ]:
(df_psm['Polyphen_Score'].isnull() & df_psm['SIFT_Score'].isnull()).sum()

In [ ]:
df_psm[df_psm['CANCER_TYPE']=='Nerve Sheath Tumor']['Center'].value_counts()

In [ ]:
df_asi[df_asi['CENTER']=='UCSF']

In [ ]:
df_asi[df_asi['CENTER']=='DFCI']

In [ ]:
df_asi[df_asi['CENTER']=='MSK']

In [ ]:
df_asi[df_asi['CENTER']=='VICC']

In [ ]:
df_dcs['SEQ_ASSAY_ID'].value_counts().sort_values().tail(20)

In [ ]:
df_psm[df_psm['CANCER_TYPE']=='Nerve Sheath Tumor']['SEQ_ASSAY_ID'].value_counts()

### Distribution of NF related quantities 

In [ ]:
df_psm.shape

In [ ]:
for gene in genie_constants.NF_HUGO_SYMBOLS:
    print("Gene: ", gene)
    
    df = df_psm[df_psm['Hugo_Symbol'].isin([gene])]
    print("  num_variants: ", df.shape[0])
    print("  num_samples:  ", df['SAMPLE_ID'].nunique())
    print("  num_patients: ", df['PATIENT_ID'].nunique())

In [ ]:
for onco in genie_constants.NF_ONCOTREE_CODES:
    print("Oncotree code: ", onco)
    
    df = df_psm[df_psm['ONCOTREE_CODE'].isin([onco])]
    print("  num_variants: ", df.shape[0])
    print("  num_samples:  ", df['SAMPLE_ID'].nunique())
    print("  num_patients: ", df['PATIENT_ID'].nunique())

# GENIE - Data CNA (Discrete Copy Number Alteration Data)

https://docs.cbioportal.org/file-formats/#discrete-copy-number-data

For each gene-sample combination, a copy number level is specified:

* "-2" is a deep loss, possibly a homozygous deletion
* "-1" is a single-copy loss (heterozygous deletion)
* "0" is diploid
* "1" indicates a low-level gain
* "2" is a high-level amplification.

In [ ]:
df_cna = genie.read_cna(syn_file_paths['data_CNA'])

In [ ]:
df_cna

### Sparsity

In [ ]:
all_cna_values = df_cna.fillna('MISSING').values.flatten()
non_null_cna_values = all_cna_values[all_cna_values!='MISSING']
non_zero_cna_values = non_null_cna_values[non_null_cna_values!=0]
set(non_zero_cna_values)

In [ ]:
cna_counts = Counter(all_cna_values)
cna_counts.most_common()

In [ ]:
df_cna_counts = pd.DataFrame(cna_counts.items(), columns=['value', 'count'])
df_cna_counts['frac'] = df_cna_counts['count'] / df_cna_counts['count'].sum()
df_cna_counts['perc'] = df_cna_counts['frac'] * 100
df_cna_counts['value_str'] = df_cna_counts['value'].astype(str)
df_cna_counts

In [ ]:
df_cna_counts[~df_cna_counts['value_str'].isin(['MISSING', '0.0'])].sum()

In [ ]:
sns.barplot(data=df_cna_counts, x='perc', y='value_str', color='grey')

### DIstribution of non-null / non-zero values

In [ ]:
_ = plt.hist(non_zero_cna_values, bins=np.linspace(-2.25, 2.25, 10))

In [ ]:
df_cna['NF1'].hist()

### Gene and Sample L2 Norms

In [ ]:
df_cna

In [ ]:
cna_gene_l2 = genie.get_cna_norms(df_cna, axis=0).sort_values()
cna_samp_l2 = genie.get_cna_norms(df_cna, axis=1).sort_values()

In [ ]:
cna_gene_l2.tail(30)

In [ ]:
cna_gene_l2["NF1"], cna_gene_l2["NF2"], cna_gene_l2["SMARCB1"], cna_gene_l2["LZTR1"]

In [ ]:
np.log10(cna_gene_l2["NF1"])

### DIstribution of Gene Vector Norms

Gene vector length is related to how many samples show copy number deviation from 0. 

In [ ]:
ax = np.log10(cna_gene_l2 + 1).hist(bins=30, figsize=(8,8))
ax.axvline(x=np.log10(cna_gene_l2["NF1"] + 1), ymin=0, ymax=1, color='red')
ax.axvline(x=np.log10(cna_gene_l2["NF2"] + 1), ymin=0, ymax=1, color='orange', ls='--')

### Distribution of Sample Vector Norms

Sample vector length is related to how many genes show copy number deviaion from 0

In [ ]:
ax = np.log10(cna_samp_l2 + 1).hist(bins=30, figsize=(8,4))

In [ ]:
ax = cna_samp_l2.hist(bins=50, figsize=(8,4))

### Melted CNA

In [ ]:
df_cna_melted = genie.get_melted_cna(df_cna)

In [ ]:
df_cna_melted

In [ ]:
cna_sentences = genie.get_cna_sentences(df_cna)

In [ ]:
cna_sentences

In [ ]:
cna_sentences.apply(lambda x: len(x)).hist(bins=50)

# How Many Samples With Mutations + CNA data? 

In [ ]:
df_psm['has_CNA'] = df_psm['SAMPLE_ID'].isin(df_cna.index)

In [ ]:
df_psm['has_CNA'].sum()

In [ ]:
df_psm.shape

In [ ]:
for gene in genie_constants.NF_HUGO_SYMBOLS:
    print("Gene: ", gene)
    
    dfv = df_psm[df_psm['Hugo_Symbol'].isin([gene])]
    dfc = dfv[dfv['has_CNA']]
    print("  num_variants: ", dfv.shape[0], dfc.shape[0])
    print("  num_samples:  ", dfv['SAMPLE_ID'].nunique(), dfc['SAMPLE_ID'].nunique())
    print("  num_patients: ", dfv['PATIENT_ID'].nunique(), dfc['PATIENT_ID'].nunique())

In [ ]:
for onco in genie_constants.NF_ONCOTREE_CODES:
    print("Oncotree code: ", onco)
    
    dfv = df_psm[df_psm['ONCOTREE_CODE'].isin([onco])]
    dfc = dfv[dfv['has_CNA']]
    print("  num_variants: ", dfv.shape[0], dfc.shape[0])
    print("  num_samples:  ", dfv['SAMPLE_ID'].nunique(), dfc['SAMPLE_ID'].nunique())
    print("  num_patients: ", dfv['PATIENT_ID'].nunique(), dfc['PATIENT_ID'].nunique())

# Data CNA seg (Segmented Copy Number Data) 

https://docs.cbioportal.org/file-formats/#segmented-data

https://cnvkit.readthedocs.io/en/stable/fileformats.html#seg

https://software.broadinstitute.org/software/igv/SEG

In [ ]:
df_seg = genie.read_cna_seg(syn_file_paths['data_cna_hg19'])

In [ ]:
df_seg

In [ ]:
df_seg['ID'].isin(df_dcs.index).all()